In [ ]:
# ==============================================================================
# 🚀 ULTIMATE CNN TEMPLATE: COLAB TURBO EDITION (EFFICIENTNET-B0)
# Optimized for: Google Colab GPU (Tesla T4)
# Features: Mixed Precision (Speedup), High Batch Size, Auto-Fix Dataset
# ==============================================================================

import tensorflow as tf
from tensorflow.keras import mixed_precision
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
import zipfile
import json
from google.colab import drive, files
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils import class_weight

In [ ]:
# ==========================================
# 0. AKTIFKAN TURBO MODE (MIXED PRECISION) ⚡
# ==========================================
# Ini bikin GPU T4 kerja 2x lebih cepat & hemat memori
try:
    policy = mixed_precision.Policy('mixed_float16')
    mixed_precision.set_global_policy(policy)
    print(f"⚡ TURBO MODE AKTIF: {policy.compute_dtype}")
except Exception as e:
    print("⚠️ Gagal aktifkan Mixed Precision (Gak masalah, lanjut standar).")

In [ ]:
# ==========================================
# 1. KONFIGURASI "MAX PERFORMANCE" 🛠️
# ==========================================
PATH_ZIP_DRIVE = '/content/drive/MyDrive/dataset_final/dataset.zip'
MODEL_NAME = 'model_klasifikasi_pro.h5'
JSON_NAME = 'labels.json'

# Settingan "Siksa Hardware"
IMG_SIZE = 260   # Resolusi Tinggi (Biar detail sisik ikan kelihatan)
BATCH_SIZE = 64  # Kita hajar 64 (GPU T4 Kuat kok!)
EPOCHS_1 = 15    # Pemanasan
EPOCHS_2 = 40    # Fine Tuning (Lama dikit biar loss 0.1)

In [ ]:
# ==========================================
# 2. SETUP & AUTO-FIX DATASET 📦
# ==========================================
print("\n🚀 MEMULAI PERSIAPAN DATASET...")

if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

base_dir = '/content/dataset_final'
if os.path.exists(base_dir): shutil.rmtree(base_dir)
os.makedirs(base_dir)

temp_dir = '/content/temp_extract'
if os.path.exists(temp_dir): shutil.rmtree(temp_dir)

print(f"📂 Mengekstrak & Merapikan: {PATH_ZIP_DRIVE}")
try:
    with zipfile.ZipFile(PATH_ZIP_DRIVE, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)

    # LOGIKA DETEKTIF PENCARI FOLDER KELAS
    real_root = None
    max_subdirs = 0
    for root, dirs, files in os.walk(temp_dir):
        valid_dirs = [d for d in dirs if not d.startswith('.')] # Abaikan folder hidden
        if len(valid_dirs) > 1:
            if len(valid_dirs) > max_subdirs:
                max_subdirs = len(valid_dirs)
                real_root = root

    if real_root:
        print(f"✅ Dataset Ditemukan di: {real_root} ({max_subdirs} Kelas)")
        for item in os.listdir(real_root):
            s = os.path.join(real_root, item)
            d = os.path.join(base_dir, item)
            if os.path.isdir(s): shutil.move(s, d)
        shutil.rmtree(temp_dir)
    else:
        raise Exception("Gagal nemu folder kelas. Cek isi ZIP lu bang.")

except Exception as e:
    print(f"❌ ERROR: {e}")
    exit()

In [ ]:
# ==========================================
# 3. DATA GENERATOR (MODE ROBOFLOW / PRE-SPLIT) 🔄
#
# PAKAI KODE INI JIKA DATASET KALIAN SUDAH TERSEDIA FOLDER Train, Test, Val.
# JIKA TIDAK ADA JANGAN PAKAI KODE INI PAKAI KODE YANG DIBAWAH!!!!!!!!!!!
# ==========================================
print("\n🔄 Menyiapkan Pipeline Data (Format Roboflow)...")

# 1. Deteksi Lokasi Folder Train & Valid
# Roboflow biasanya strukturnya: /dataset/train dan /dataset/valid (atau val)
train_dir = os.path.join(base_dir, 'train')

# Cek nama folder validasi (kadang 'valid', kadang 'val')
val_dir = os.path.join(base_dir, 'valid')
if not os.path.exists(val_dir):
    val_dir = os.path.join(base_dir, 'val')

# Cek apakah folder beneran ada
if not os.path.exists(train_dir) or not os.path.exists(val_dir):
    print("❌ ERROR: Gak nemu folder 'train' atau 'valid'/'val'!")
    print(f"   Isi folder saat ini: {os.listdir(base_dir)}")
    # Fallback darurat: kalau ternyata strukturnya berantakan, script ini akan stop
    raise Exception("Struktur folder tidak sesuai format Roboflow Train/Val")

print(f"📍 Folder Latihan: {train_dir}")
print(f"📍 Folder Ujian:   {val_dir}")

# 2. Siapkan Generator (TANPA VALIDATION SPLIT)
# Kita hapus 'validation_split' karena data sudah terpisah fisik
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# 3. Load Data
print("\n📥 Loading Data Latihan...")
train_data = train_datagen.flow_from_directory(
    train_dir,            # <--- Langsung tembak folder TRAIN
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

print("\n📥 Loading Data Validasi...")
val_data = val_datagen.flow_from_directory(
    val_dir,              # <--- Langsung tembak folder VALID
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
# ==========================================
# 3. DATA GENERATOR (FULL SPEED) 🔄
# ==========================================
print("\n🔄 Menyiapkan Pipeline Data...")

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

# Gunakan workers=4 buat manfaatin semua Core CPU Colab
train_data = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_data = train_datagen.flow_from_directory(
    base_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

In [ ]:
# ==========================================
# 4. CLASS WEIGHTS (KEADILAN) ⚖️
# ==========================================
print("\n⚖️ Menghitung Bobot Penyeimbang...")
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
# ==========================================
# 5. MEMBANGUN MODEL (ARSITEKTUR PRO) 🧠
# ==========================================
print("\n🏗️ Membangun Arsitektur EfficientNetB0...")

base_model = EfficientNetB0(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)

# Head Model yang Agresif
x = BatchNormalization()(x)
x = Dropout(0.4)(x) # Dropout agak gede biar gak cepet overfit pas ngebut
x = Dense(512, activation='relu')(x) # Layer tebal biar pinter
x = BatchNormalization()(x)
x = Dropout(0.4)(x)

# PENTING: dtype='float32' wajib di output layer saat pakai Mixed Precision
outputs = Dense(train_data.num_classes, activation='softmax', dtype='float32')(x)

model = tf.keras.Model(inputs, outputs)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7, verbose=1)
]

In [ ]:
# ==========================================
# 6. TRAINING TAHAP 1 (WARM UP) 🥊
# ==========================================
print(f"\n🔥 MULAI TRAINING TAHAP 1 (Frozen - {EPOCHS_1} Epochs)...")

model.compile(
    optimizer=Adam(learning_rate=0.001, clipnorm=1.0),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

history1 = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS_1,
    callbacks=callbacks,
    class_weight=class_weights_dict,
    workers=4, # Pakai 4 core CPU buat loading data
    use_multiprocessing=True
)

In [ ]:
# ==========================================
# 7. TRAINING TAHAP 2 (FINE TUNING) 🚀
# ==========================================
print(f"\n❄️ MULAI TRAINING TAHAP 2 (Unfreeze - {EPOCHS_2} Epochs)...")
base_model.trainable = True

# Label Smoothing buat ngejar loss rendah (0.1 - 0.2)
loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)

model.compile(
    optimizer=Adam(learning_rate=1e-5, clipnorm=1.0),
    loss=loss_fn,
    metrics=['accuracy']
)

history2 = model.fit(
    train_data,
    validation_data=val_data,
    epochs=EPOCHS_2,
    callbacks=callbacks,
    class_weight=class_weights_dict,
    workers=4,
    use_multiprocessing=True
)

In [ ]:
# ==========================================
# 8. SIMPAN & DOWNLOAD OTOMATIS 💾
# ==========================================
print("\n💾 Menyimpan Hasil Kerja Keras...")

# Save Model
model.save(MODEL_NAME)

# Save JSON
kamus_label = {str(v): k.replace("_", " ").upper() for k, v in train_data.class_indices.items()}
with open(JSON_NAME, 'w') as f:
    json.dump(kamus_label, f, indent=4)

# Plot Grafik
acc = history1.history['accuracy'] + history2.history['accuracy']
val_acc = history1.history['val_accuracy'] + history2.history['val_accuracy']
loss = history1.history['loss'] + history2.history['loss']
val_loss = history1.history['val_loss'] + history2.history['val_loss']

plt.figure(figsize=(15, 6))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Akurasi Dewa')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Loss (Target 0.1)')
plt.legend()
plt.grid(True)
plt.savefig('grafik_keren.png') # Simpan grafik juga
plt.show()

print("🎉 MISSION ACCOMPLISHED! Download file dimulai...")
files.download(MODEL_NAME)
files.download(JSON_NAME)
files.download('grafik_keren.png')